In [1]:
import pandas as pd
import numpy as np
import os

The data for England and Wales (as well as sub-regions, such as London) can be accessed from the Office of National Statistics here: https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales

We use these data as the official counts of the total number of deaths as well as the total number of Covid-19 deaths.

Note that the ONS Covid deaths are slightly higher than those reported on GOV.UK here: https://www.gov.uk/government/publications/covid-19-track-coronavirus-cases

The reason for using ONS counts for both is that both total deaths and covid-19 deaths are based on **date of registration** of the death. As we want to make apples-to-apples comparisons as far as possible, we are not using covid deaths from .GOV.UK.

The formatting of the ONS data as they are distributed is somewhat inconvenient, which is why we have to process them first. You can skip this step and directly work with the processed files, which can be found in `../uk/data/processed`.

**IMPORTANT**
The upside of the somewhat difficult to process ONS data is that they illustrate some of the complexities associated with real time mortality reporting. It is worth going into this at some length.

- The ONS weekly figures for all-cause mortality are by week of **registration** of the death. This is not the same as the date/week the death **occurred**. The reason for the difference are registration delays. 

- ONS also publishes, for Covid-19 only, a series of daily and weekly **occurrences** of death. This would be preferable to use for our purposes (since it is based on the date the death actually happened). It is only available, however, for Covid-19 and not for total deaths. To make apples-to-apples comparisons, **we are using registered, not occurred deaths**. 

- One worry with this approach could be that registration delay may be especially severe during times such as Covid-19 when the administrative system can become overwhelmed. 

- A slightly distinct worry would be if Covid-19 deaths and other deaths suffer from differential degree of registration delay. 

- As we do in other places, we therefore repeat our plea to **interpret our results with caution**.

In [2]:
wdir = '../data'
outdir = wdir + '/processed'

In [3]:
def dump_excel_to_df(excel_file, sheet_name, skip_rows, last_row, respitory_name, n_weeks, delete_row1=False, custom_keepcols=None):
    #Read the Excel file and traspose it
    df = pd.read_excel(wdir + '/{}'.format(excel_file),
                   sheet_name=sheet_name, 
                   skiprows=skip_rows).T
    #Keep data up to last row last_row specified
    df = df.iloc[:,0:last_row].copy(deep=True)
    #Get the columns names, the first row
    cols = list(df.iloc[0,:])
    #Delete the first row element from df
    df = df.iloc[1:,:].copy(deep=True)
    #Name the column with the cols variable
    df.columns = cols
    
    #If there is twice the name of the variable (it is in some years, e.g. 2019), delete it
    if delete_row1:
        df = df.iloc[1:,:].reset_index(drop=True)
    
    df['week'] = np.arange(1,n_weeks+1)
    
    if custom_keepcols:
        keepcols = custom_keepcols 
    else:
        keepcols = ['week',
                    'Week ended',
                    'Total deaths, all ages',
                    'Total deaths: average of corresponding',
                    respitory_name,
                    'North East',
                    'North West',
                    'Yorkshire and The Humber',
                    'East Midlands',
                    'West Midlands',
                    'East',
                    'London',
                    'South East',
                    'South West',
                    'Wales']   
    df = df[keepcols].reset_index(drop=True)
    
    return df

In [4]:
xlslist = ['publishedweek2010.xls',
           'publishedweek2011.xls',
           'publishedweek2012.xls',
           'publishedweek2013.xls', 
           'publishedweek2014.xls', 
           'publishedweek2015.xls', 
           'publishedweek522016.xls', 
           'publishedweek522017.xls', 
           'publishedweek522018withupdatedrespiratoryrow.xls', 
           'publishedweek522019.xls', 
           'publishedweek122020.xlsx', 
           'publishedweek122020.xlsx',
           'publishedweek1320201.xlsx',
           'publishedweek1320201.xlsx', 
           'referencetablesweek142020.xlsx',
           'referencetablesweek142020.xlsx']

sheet_list = ['Weekly Figures 2010',
              'Weekly Figures 2011',
              'Weekly Figures 2012',
              'Weekly Figures 2013', 
              'Weekly Figures 2014', 
              'Weekly Figures 2015', 
              'Weekly figures 2016', 
              'Weekly figures 2017', 
              'Weekly figures 2018', 
              'Weekly figures 2019',
              'Weekly figures 2020', 
              'Covid-19 - Weekly figures 2020',
              'Weekly figures 2020',
              'Covid-19 - Weekly figures 2020',
              'Weekly figures 2020',
              'Covid-19 - Weekly registrations']

skip_list = [3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4]

last_row_list = [48, 49, 48, 48, 49, 48, 48, 48, 48, 48, 50, 50, 50, 50, 87, 81]

weeks_list = [52, 52, 52, 52, 52, 53, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53]

respitory_list = ['All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2001',
                  'ICD-10 v 2010',
                  'All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2010', 
                  'All respiratory diseases (ICD-10 J00-J99)  ICD-10 v 2010',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)',
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'All respiratory diseases (ICD-10 J00-J99) ICD-10 v 2013 (IRIS)', 
                  'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)', 
                  '',
                  'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)',
                  '',
                  'Deaths where the underlying cause was respiratory disease (ICD-10 J00-J99)', 
                  '']

yrs = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, '2020_covid', 2020, '2020_covid', 2020, '2020_covid']
keys = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       '2020_wk12', '2020_wk12_covid', '2020_wk13', '2020_wk13_covid', '2020_wk14', '2020_wk14_covid']

delete_list = [False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True]


print(len(xlslist),
      len(sheet_list),
      len(skip_list),
      len(last_row_list),
      len(weeks_list),
      len(respitory_list),
      len(yrs),
      len(keys),
      len(delete_list))

16 16 16 16 16 16 16 16 16


In [5]:
idx = 15

custcols = ['week',
            'Week ended',
            'Deaths involving COVID-19, all ages1',
            'North East',
            'North West',
            'Yorkshire and The Humber',
            'East Midlands',
            'West Midlands',
            'East',
            'London',
            'South East',
            'South West',
            'Wales'] 

# custcols=None

df = dump_excel_to_df(xlslist[idx],
                      sheet_list[idx],
                      skip_rows=skip_list[idx],
                      last_row=last_row_list[idx],
                      respitory_name=respitory_list[idx], 
                      n_weeks=weeks_list[idx], 
                      delete_row1=delete_list[idx],
                      custom_keepcols=custcols)

# df

In [6]:
df_dict = {}

In [7]:
for idx in range(16):
    
    if yrs[idx] == '2020_covid':
        
        custcols = ['week',
                    'Week ended',
                    'Deaths involving COVID-19, all ages1',
                    'North East',
                    'North West',
                    'Yorkshire and The Humber',
                    'East Midlands',
                    'West Midlands',
                    'East',
                    'London',
                    'South East',
                    'South West',
                    'Wales'] 
    else:
        
        custcols = None
    
    df = dump_excel_to_df(xlslist[idx],
                          sheet_list[idx],
                          skip_rows=skip_list[idx],
                          last_row=last_row_list[idx],
                          respitory_name=respitory_list[idx], 
                          n_weeks=weeks_list[idx], 
                          delete_row1=delete_list[idx],
                          custom_keepcols=custcols)

    df_dict[keys[idx]] = df

In [8]:
df_dict.keys()

dict_keys([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, '2020_wk12', '2020_wk12_covid', '2020_wk13', '2020_wk13_covid', '2020_wk14', '2020_wk14_covid'])

In [9]:
for key, df in df_dict.items():
    
    if 'covid' in str(key):
    
        colnames = ['week',
                    'week_ended',
                    'covid_deaths',
                    'north_east',
                    'north_west',
                    'yorkshire_and_the_humber',
                    'east_midlands',
                    'west_midlands',
                    'east',
                    'london',
                    'south_east',
                    'south_west',
                    'wales'] 
        
    else:
        
         colnames = ['week',
                     'week_ended',
                     'total_deaths_all_ages',
                     'total_deaths_avg_corresponding_5',
                     'respitory_deaths',
                     'north_east',
                     'north_west',
                     'yorkshire_and_the_humber',
                     'east_midlands',
                     'west_midlands',
                     'east',
                     'london',
                     'south_east',
                     'south_west',
                     'wales'] 
    
    
    df.columns = colnames
    
    
    df.to_csv(outdir + '/deaths_{}.csv'.format(key), index=False)